In [ ]:
"""
Created on Friday Aug 30 

@author: krish patel

"""

import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to draw dense optical flow
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T

    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)
 
    return img_bgr

# Initialize video capture and select initial ROIs
def OnInit(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video stream or file")
        quit()

    ret, frame = cap.read()
    if not ret:
        print("Error reading the first frame.")
        quit()

    x, y, w, h = cv2.selectROI("Select ROI for content flow", frame)
    x_scale, y_scale, w_scale, h_scale = cv2.selectROI("Select ROI for scale (1 cm)", frame)
    cv2.destroyAllWindows()

    return frame[:, :, 0], cap, (x, y, w, h), (x_scale, y_scale, w_scale, h_scale), cap.get(cv2.CAP_PROP_FPS)

# Open the video file
video_path = r"C:\Users\chenhuizingalab\Desktop\COLON_ANALYSIS_CODE\previous_experiments_and_results\new_videos_to_run\E32_ASC_B_DB_F_C4/E32_ASC_B_DB_F_C4.avi"  # Replace with the path to your video file

old_frame_gray, cap, roi_content, roi_scale, fps = OnInit(video_path)
x, y, w, h = roi_content
x_scale, y_scale, w_scale, h_scale = roi_scale

# Calculate pixels per cm
pixels_per_cm = w_scale / 1.0  # Assuming the selected width is 1 cm

# Initialize variables for motion detection
prev_frame = None
motion_directions = []
motion_threshold = 0.5  # Threshold to filter out minor movements

# Iterate through the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Break the loop at the end of the video

    # Crop the current ROI
    frame_roi = frame[y:y+h, x:x+w]

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame_roi, cv2.COLOR_BGR2GRAY)

    # Motion detection using dense optical flow
    if prev_frame is not None:
        # Calculate optical flow within the ROI
        flow = cv2.calcOpticalFlowFarneback(prev_frame, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Extract horizontal flow (fx) and ignore vertical flow (fy)
        fx, fy = flow[..., 0], flow[..., 1]
        fy[:] = 0

        # Calculate motion magnitude and filter based on threshold
        motion_magnitude = np.sqrt(fx**2 + fy**2)
        significant_motion = motion_magnitude > motion_threshold

        # Calculate mean horizontal motion direction
        if np.any(significant_motion):
            mean_motion_direction = np.mean(fx[significant_motion])
        else:
            mean_motion_direction = 0

        # Record the mean motion direction in mm
        motion_directions.append((mean_motion_direction / pixels_per_cm) * 1000)  # Convert to mm

        # Visualize the dense optical flow within the ROI
        flow_image_roi = draw_flow(gray_frame, flow)

        # Draw the flow on the whole frame with the ROI
        flow_image_full = frame.copy()
        flow_image_full[y:y+h, x:x+w] = flow_image_roi
        cv2.rectangle(flow_image_full, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.imshow('Dense Optical Flow', flow_image_full)

    prev_frame = gray_frame

    # Display the frame with motion direction arrows
    cv2.imshow('Frame', frame_roi)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit the loop

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()

# Plot motion directions over time
plt.figure(figsize=(20, 10))
time_points = np.arange(len(motion_directions))

# Compute the time values for each frame
time_values = np.arange(len(motion_directions)) / fps

# Define the window size for the moving average
window_size = 200  # Adjust this based on your preference

# Compute the moving average
smoothed_motion_direction = np.convolve(motion_directions, np.ones(window_size)/window_size, mode='same')

# Plotting the smoothed line plot
plt.plot(time_values, smoothed_motion_direction, linestyle='-', linewidth=2, color='k')

# Remove x-axis label, ticks, and values
plt.xlabel('')  # No x-axis label
plt.xticks([])  # No x-axis ticks
plt.grid(False)  # No grid

# Annotate the plot to indicate motion direction
plt.fill_between(time_values, 0, smoothed_motion_direction, where=smoothed_motion_direction >= 0, color='black', alpha=0.3, label='Right motion')
plt.fill_between(time_values, 0, smoothed_motion_direction, where=smoothed_motion_direction < 0, color='black', alpha=0.3, label='Left motion')

# Add a thicker line at y=0 for emphasis
plt.axhline(0, color='k', linewidth=3)  # Increased linewidth to 3

# Move y-axis to the right side
ax = plt.gca()
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")

# Set the y-axis label with the desired properties
ax.set_ylabel('Flow Direction Magnitude', fontsize=28, labelpad=20, rotation=-90, va='center')

# Adjust y-axis tick label size
ax.tick_params(axis='y', labelsize=35)

# Remove the x-axis ticks
#ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

# Remove the top and left spines
#ax.spines['top'].set_visible(False)
#ax.spines['left'].set_visible(False)

# Show the plot
plt.show()

# Plotting the smoothed line plot without labels, legends, or grid
fig, ax = plt.subplots(figsize=(20, 10), dpi=100)
ax.plot(time_values, smoothed_motion_direction, linestyle='-', linewidth=2, color='k')

# Annotate the plot to indicate motion direction with transparent shading
ax.fill_between(time_values, 0, smoothed_motion_direction, where=smoothed_motion_direction >= 0, color='black', alpha=0.3)
ax.fill_between(time_values, 0, smoothed_motion_direction, where=smoothed_motion_direction < 0, color='black', alpha=0.3)

# Add a thicker line at y=0 for emphasis
ax.axhline(0, color='k', linewidth=3)  # Increased linewidth to 3

# Move y-axis to the right side
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")

# Set the y-axis label with the desired properties
ax.set_ylabel('Flow Direction Magnitude', fontsize=28, labelpad=20, rotation=-90, va='center')

# Adjust y-axis tick label size
ax.tick_params(axis='y', labelsize=35)

# Remove the x-axis ticks
ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)

# Remove the top and left spines
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Save the plot as a PNG with a transparent background
fig.savefig('C:/Users/chenhuizingalab/Desktop/new_videos_to_run/E21_C11_DESC_SUPP/E21_C11_DESC_SUPP_good_flow_any.png', transparent=True, bbox_inches='tight', pad_inches=0)

# Close the figure to free up memory
plt.close(fig)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Define a threshold to separate motor patterns from breathing
threshold = 7.5  # Example threshold value in mm

# Segment the signal into motor patterns and breathing based on the threshold
motor_patterns = np.where(np.abs(smoothed_motion_direction) > threshold, smoothed_motion_direction, 0)
breathing = np.where(np.abs(smoothed_motion_direction) <= threshold, smoothed_motion_direction, 0)

# Calculate the area under the curve (sum of absolute values) for each segment
area_motor_patterns = np.sum(np.abs(motor_patterns))
area_breathing = np.sum(np.abs(breathing))
total_area = area_motor_patterns + area_breathing

# Calculate the percentages
percentage_motor_patterns = (area_motor_patterns / total_area) * 100
percentage_breathing = (area_breathing / total_area) * 100

# Output the results
print(f"Percentage of content flow due to motor patterns: {percentage_motor_patterns:.2f}%")
print(f"Percentage of content flow due to breathing: {percentage_breathing:.2f}%")

# Plot the motor patterns separately
plt.figure(figsize=(12, 6))
plt.fill_between(range(len(smoothed_motion_direction)), motor_patterns, 0,
                 where=(motor_patterns != 0), color='red', alpha=0.5, label='Motor Patterns Area')
plt.plot(smoothed_motion_direction, label='Original Signal', color='black', alpha=0.3)
plt.axhline(y=threshold, color='green', linestyle=':', label='Threshold')
plt.axhline(y=-threshold, color='green', linestyle=':')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Flow Direction Magnitude (mm)')
plt.title('Motor Patterns in Flow Signal with Highlighted Area Under the Curve')
plt.show()

# Plot the breathing separately
plt.figure(figsize=(12, 6))
plt.fill_between(range(len(smoothed_motion_direction)), breathing, 0,
                 where=(breathing != 0), color='blue', alpha=0.5, label='Breathing Area')
plt.plot(smoothed_motion_direction, label='Original Signal', color='black', alpha=0.3)
plt.axhline(y=threshold, color='green', linestyle=':', label='Threshold')
plt.axhline(y=-threshold, color='green', linestyle=':')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Flow Direction Magnitude (mm)')
plt.title('Breathing in Flow Signal with Highlighted Area Under the Curve')
plt.show()
